In [ ]:
from google.oauth2.service_account import Credentials
from googleapiclient.discovery import build
import os
import json
import io
from googleapiclient.http import MediaIoBaseDownload
import pandas as pd
from datetime import datetime
from modules.cleaning_functions import json_to_dataframe

In [7]:
# Chargement des variables d'environement
account_file = os.getenv("SERVICE_ACCOUNT_FILE")
folder_id = os.getenv("FOLDER_ID")

In [24]:
# Authentification
creds = Credentials.from_service_account_file(account_file, scopes=['https://www.googleapis.com/auth/drive.readonly'])
service = build('drive', 'v3', credentials=creds)
counter = 0
# Requête pour lister les fichiers dans le dossier
query = f"'{folder_id}' in parents"
results_list = []
token = ''
while token is not None : 
    results = service.files().list(
        q=query,
        #pageSize=50,  # Nombre max de fichiers à récupérer
        pageToken = token
    ).execute()
    # Liste des fichiers
    items = results.get('files', [])
    token = results.get('nextPageToken', None)
    results_list.extend(items)

In [ ]:
# date de première collecte Arezki
date_ref = datetime.strptime("2025-02-18 20:52:00.000", "%Y-%m-%d %H:%M:%S.%f")

try :
    # Affichage
    if not results_list:
        print("Aucun fichier trouvé dans le dossier.")
    else:
                
        df = pd.DataFrame()

        for item in results_list:
            
            ## 1 - Analyser la date et l'heure du fichier
            file_id = item["id"] 
            file_name = item["name"].strip()
            
            # Extraire la partie date et heure
            date_part = file_name.split('_')[1] 
            time_part = file_name.split('_')[2].replace('.txt', '')  

            # Nettoyer l'heure
            time_part = time_part.replace('-', ':')

            # Combiner date et heure dans le bon format
            datetime_str = f"{date_part} {time_part}"

            # Parser la chaîne en datetime
            timestamp = datetime.strptime(datetime_str.strip(), "%Y-%m-%d %H:%M")
            
            if timestamp <= date_ref :
                
                request = service.files().get_media(fileId=file_id)
                fh = io.BytesIO()
                downloader = MediaIoBaseDownload(fh, request)
                done = False
                
                while not done:
                    status, done = downloader.next_chunk()
                    print("Téléchargement en cours... %.2f%%" % (status.progress() * 100))

                fh.seek(0)  # Revenir au début du fichier
                
                # Charger le JSON et transformer en DataFrame
                json_data = json.load(fh)
                records = json_data['records']
                for record in records :
                    df_intermediaire = json_to_dataframe([record['fields']])#pd.DataFrame([record['fields']]) 
                    df_intermediaire['id_technique'] = df_intermediaire.apply(lambda x : str(x['id'])+'-'+str(x['horodatage']),axis=1)
                    df = pd.concat([df, df_intermediaire])
                
except Exception as e :
    print(str(e))
                        
df.to_csv('trafic_data_make2.csv', index = False)

Fichiers dans le dossier :
Téléchargement en cours... 100.00%
Téléchargement en cours... 100.00%
Téléchargement en cours... 100.00%
Téléchargement en cours... 100.00%
Téléchargement en cours... 100.00%
Téléchargement en cours... 100.00%
Téléchargement en cours... 100.00%
Téléchargement en cours... 100.00%
Téléchargement en cours... 100.00%
Téléchargement en cours... 100.00%
Téléchargement en cours... 100.00%
Téléchargement en cours... 100.00%
Téléchargement en cours... 100.00%
Téléchargement en cours... 100.00%
Téléchargement en cours... 100.00%
Téléchargement en cours... 100.00%
Téléchargement en cours... 100.00%
Téléchargement en cours... 100.00%
Téléchargement en cours... 100.00%
Téléchargement en cours... 100.00%
Téléchargement en cours... 100.00%
Téléchargement en cours... 100.00%
Téléchargement en cours... 100.00%
Téléchargement en cours... 100.00%
Téléchargement en cours... 100.00%
Téléchargement en cours... 100.00%
Téléchargement en cours... 100.00%
Téléchargement en cours... 1

# Check data

In [30]:
df_make = pd.read_csv('trafic_data_make.csv')
df_make.head()

,id,debit,longueur,taux_occupation,code_couleur,nom_du_troncon,etat_du_trafic,temps_de_parcours,vitesse,geo_point_2d,geometrie,shape_geo,horodatage,type_geo,coordinates_geo,id_technique
0,772,180,410,2.5,3,Vannes I9,Fluide,58,25,"{47.23414730688424,-1.5808786419612229}","{{-1.583123434893785,47.235185973808406},{-1.5...",LineString,2025-02-18 20:43:00.000,Point,"{-1.5808786419612229,47.23414730688424}",772-2025-02-18 20:43:00.000
1,9,180,309,1.6,3,Schuman I5,Fluide,53,21,"{47.23448297241201,-1.567120540210308}","{{-1.568160939861017,47.23568134656465},{-1.56...",LineString,2025-02-18 20:43:00.000,Point,"{-1.567120540210308,47.23448297241201}",9-2025-02-18 20:43:00.000
2,5444,60,177,0.8,3,Pasteur P1,Fluide,53,12,"{47.21255561611825,-1.579953397082114}","{{-1.581033846171769,47.21286509290865},{-1.57...",LineString,2025-02-18 20:43:00.000,Point,"{-1.579953397082114,47.21255561611825}",5444-2025-02-18 20:43:00.000
3,5074,60,204,0.4,3,Pitre Chevallier I1,Fluide,52,14,"{47.22345227234604,-1.551030243214234}","{{-1.552016684048891,47.222822066240326},{-1.5...",LineString,2025-02-18 20:43:00.000,Point,"{-1.551030243214234,47.22345227234604}",5074-2025-02-18 20:43:00.000
4,5053,60,224,0.8,3,Barbusse P2,Fluide,43,19,"{47.22705355840811,-1.5524880999813486}","{{-1.551553912867712,47.22760386550291},{-1.55...",LineString,2025-02-18 20:43:00.000,Point,"{-1.5524880999813486,47.22705355840811}",5053-2025-02-18 20:43:00.000


In [31]:
len(df_make)

118860

In [33]:
df_make['horodatage'].min()

'2025-02-16 20:22:00.000'